In [24]:
# Import required libraries into the jupyter notebook

import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np # linear algebra

Beginning Code blocks to do the things I want it to do

In [25]:
# Step to read in weather data and pizza data csv files from the data directory
weather_file = pd.read_csv("data/alexandria_weather_data.csv", low_memory=False)
pizza_file = pd.read_csv("data/alexandria_pizza_data.csv", low_memory=False)

# This will receive warnings. I could have selected only the columns I needed, but cleaning data is a part of the assignment and I will do that in cells below.


In [26]:
# Display head of weather file to show successful read of file
weather_file.head()

,name,datetime,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,...,solarenergy,uvindex,severerisk,sunrise,sunset,moonphase,conditions,description,icon,stations
0,41001,3/1/2021,55,32,42,55.0,23.9,35.3,28.8,62.0,...,12.9,6,NaN,2021-03-01T07:09:32,2021-03-01T18:30:39,0.58,"Rain, Partially cloudy",Clearing in the afternoon with early morning r...,rain,"KI69,KLUK,E5157,72421093814,KCVG,72429793812,A..."
1,41001,3/2/2021,44,24,33,44.4,16.6,28.6,16.5,52.2,...,17.1,7,NaN,2021-03-02T07:08:04,2021-03-02T18:31:42,0.62,Clear,Clear conditions throughout the day.,clear-day,"KI69,F8613,KLUK,E5157,72421093814,KCVG,7242979..."
2,41001,3/3/2021,62,27,44,62.1,22.7,42.2,26.6,54.2,...,16.6,7,NaN,2021-03-03T07:06:36,2021-03-03T18:32:45,0.65,Clear,Clear conditions throughout the day.,clear-day,"KI69,F8613,KLUK,E5157,72421093814,KCVG,7242979..."
3,41001,3/4/2021,52,30,41,51.9,23.2,36.7,25.9,57.3,...,16.9,7,NaN,2021-03-04T07:05:07,2021-03-04T18:33:47,0.69,Clear,Clear conditions throughout the day.,clear-day,"KI69,F8613,KLUK,E5157,72421093814,KCVG,7242979..."
4,41001,3/5/2021,47,25,35,42.2,18.4,31.0,18.9,54.5,...,17.2,7,NaN,2021-03-05T07:03:38,2021-03-05T18:34:50,0.75,Clear,Clear conditions throughout the day.,clear-day,"KI69,KLUK,E5157,72421093814,KCVG,72429793812,A..."


In [27]:
# Diesply head of pizza date file to show successful read of file
pizza_file.head()

,BusDate,DayofWeek,OrderID,OtotSubTotal,OrdServiceType,OrdVoidDate,OrdTaxExemptID,OrdOnlineOrderID,PmtType
0,3/1/2021,Monday,3,19.60,1,NaN,NaN,NaN,3
1,3/1/2021,Monday,2,20.74,3,NaN,NaN,NaN,3
2,3/1/2021,Monday,5,20.97,1,NaN,NaN,NaN,3
3,3/1/2021,Monday,6,9.99,1,NaN,NaN,NaN,3
4,3/1/2021,Monday,10,26.27,1,NaN,NaN,NaN,3


In [28]:
# Add steps in here to round the temp to the nearest ten degrees


weather_file.temp = weather_file.temp.round(-1) 
weather_file.head()

,name,datetime,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,...,solarenergy,uvindex,severerisk,sunrise,sunset,moonphase,conditions,description,icon,stations
0,41001,3/1/2021,55,32,40,55.0,23.9,35.3,28.8,62.0,...,12.9,6,NaN,2021-03-01T07:09:32,2021-03-01T18:30:39,0.58,"Rain, Partially cloudy",Clearing in the afternoon with early morning r...,rain,"KI69,KLUK,E5157,72421093814,KCVG,72429793812,A..."
1,41001,3/2/2021,44,24,30,44.4,16.6,28.6,16.5,52.2,...,17.1,7,NaN,2021-03-02T07:08:04,2021-03-02T18:31:42,0.62,Clear,Clear conditions throughout the day.,clear-day,"KI69,F8613,KLUK,E5157,72421093814,KCVG,7242979..."
2,41001,3/3/2021,62,27,40,62.1,22.7,42.2,26.6,54.2,...,16.6,7,NaN,2021-03-03T07:06:36,2021-03-03T18:32:45,0.65,Clear,Clear conditions throughout the day.,clear-day,"KI69,F8613,KLUK,E5157,72421093814,KCVG,7242979..."
3,41001,3/4/2021,52,30,40,51.9,23.2,36.7,25.9,57.3,...,16.9,7,NaN,2021-03-04T07:05:07,2021-03-04T18:33:47,0.69,Clear,Clear conditions throughout the day.,clear-day,"KI69,F8613,KLUK,E5157,72421093814,KCVG,7242979..."
4,41001,3/5/2021,47,25,40,42.2,18.4,31.0,18.9,54.5,...,17.2,7,NaN,2021-03-05T07:03:38,2021-03-05T18:34:50,0.75,Clear,Clear conditions throughout the day.,clear-day,"KI69,KLUK,E5157,72421093814,KCVG,72429793812,A..."


In [29]:
# Cleaning data ahead of the merge since there were column warnings above

pizza_file.drop(['OrdVoidDate', 'OrdTaxExemptID', 'OrdOnlineOrderID', 'PmtType'],axis=1,inplace=True)
weather_file.drop(['name', 'icon', 'stations', 'sunrise', 'sunset', 'moonphase', 'description'],axis=1,inplace=True)

pizza_file.head()

,BusDate,DayofWeek,OrderID,OtotSubTotal,OrdServiceType
0,3/1/2021,Monday,3,19.60,1
1,3/1/2021,Monday,2,20.74,3
2,3/1/2021,Monday,5,20.97,1
3,3/1/2021,Monday,6,9.99,1
4,3/1/2021,Monday,10,26.27,1


In [30]:
weather_file.head()

,datetime,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,precip,...,windspeed,winddir,sealevelpressure,cloudcover,visibility,solarradiation,solarenergy,uvindex,severerisk,conditions
0,3/1/2021,55,32,40,55.0,23.9,35.3,28.8,62.0,0.031,...,16.7,319.2,1019.8,47.7,9.7,152.6,12.9,6,NaN,"Rain, Partially cloudy"
1,3/2/2021,44,24,30,44.4,16.6,28.6,16.5,52.2,0.000,...,8.9,49.6,1026.0,0.5,9.9,197.2,17.1,7,NaN,Clear
2,3/3/2021,62,27,40,62.1,22.7,42.2,26.6,54.2,0.000,...,13.6,232.9,1017.5,0.0,9.9,194.3,16.6,7,NaN,Clear
3,3/4/2021,52,30,40,51.9,23.2,36.7,25.9,57.3,0.000,...,16.4,358.6,1021.9,1.3,9.9,197.0,16.9,7,NaN,Clear
4,3/5/2021,47,25,40,42.2,18.4,31.0,18.9,54.5,0.000,...,11.3,357.0,1026.0,7.7,9.9,201.1,17.2,7,NaN,Clear


In [31]:
# Joining both files into one data element to begin analysis

left = pizza_file.set_index('BusDate')
right = weather_file.set_index('datetime')

combined_dataset = left.join(right, lsuffix='_p', rsuffix="_w")
combined_dataset.head()



,DayofWeek,OrderID,OtotSubTotal,OrdServiceType,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,...,windspeed,winddir,sealevelpressure,cloudcover,visibility,solarradiation,solarenergy,uvindex,severerisk,conditions
BusDate,,,,,,,,,,,,,,,,,,,,,
1/1/2022,Saturday,2,19.74,3,65,52,60,64.6,52.1,56.9,...,14.8,227.7,1004.9,99.9,6.7,8.6,0.8,0,NaN,"Rain, Overcast"
1/1/2022,Saturday,3,17.99,3,65,52,60,64.6,52.1,56.9,...,14.8,227.7,1004.9,99.9,6.7,8.6,0.8,0,NaN,"Rain, Overcast"
1/1/2022,Saturday,4,33.98,3,65,52,60,64.6,52.1,56.9,...,14.8,227.7,1004.9,99.9,6.7,8.6,0.8,0,NaN,"Rain, Overcast"
1/1/2022,Saturday,5,46.51,1,65,52,60,64.6,52.1,56.9,...,14.8,227.7,1004.9,99.9,6.7,8.6,0.8,0,NaN,"Rain, Overcast"
1/1/2022,Saturday,6,12.00,1,65,52,60,64.6,52.1,56.9,...,14.8,227.7,1004.9,99.9,6.7,8.6,0.8,0,NaN,"Rain, Overcast"


In [32]:
# This cell will be used to try to sample the data and show various counts

orders_per_avgtemp = combined_dataset.temp.value_counts()
orders_per_avgtemp

temp
70    23761
60    17951
80    14784
50    14123
40    14014
30     6726
20     1927
10      124
0        76
Name: count, dtype: int64

In [33]:
# This cell will be used to try to sample the data and show various counts

orders_per_maxtemp = combined_dataset.tempmax.value_counts()
orders_per_maxtemp

tempmax
82    3473
83    3260
84    3133
78    2923
75    2748
      ... 
17     138
24     128
29     128
30     119
14      76
Name: count, Length: 72, dtype: int64

In [34]:
# This cell will be used to try to sample the data and show various counts

orders_per_mintemp = combined_dataset.tempmin.value_counts()
orders_per_mintemp

tempmin
 59    3101
 62    2932
 57    2871
 70    2722
 64    2453
       ... 
 10     138
 76     132
 3      124
-7       76
 6       66
Name: count, Length: 69, dtype: int64

In [35]:
# This cell will be used to try to sample the data and show various counts

orders_per_day = combined_dataset.DayofWeek.value_counts()
orders_per_day

DayofWeek
Friday       20113
Saturday     18462
Sunday       13451
Thursday     11603
Wednesday    10712
Tuesday       9945
Monday        9200
Name: count, dtype: int64

In [36]:
# Creating dataframes for weekend and M-Th pizza sales 

# Weekend
friday_select = (combined_dataset.DayofWeek == 'Friday')
friday_orders = combined_dataset[friday_select]
saturday_select = (combined_dataset.DayofWeek == 'Saturday')
saturday_orders = combined_dataset[saturday_select]
sunday_select = (combined_dataset.DayofWeek == 'Sunday')
sunday_orders = combined_dataset[sunday_select]
weekend_orders = pd.concat([friday_orders, saturday_orders, sunday_orders])

# Weekday
monday_select = (combined_dataset.DayofWeek == "Monday")
monday_orders = combined_dataset[monday_select]
tuesday_select = (combined_dataset.DayofWeek == "Tuesday")
tuesday_orders = combined_dataset[tuesday_select]
wednesday_select = (combined_dataset.DayofWeek == "Wednesday")
wednesday_orders = combined_dataset[wednesday_select]
thursday_select = (combined_dataset.DayofWeek == "Thursday")
thursday_orders = combined_dataset[thursday_select]
montues_orders = pd.concat([monday_orders, tuesday_orders])
weekday_orders = pd.concat([monday_orders, tuesday_orders, wednesday_orders, thursday_orders])

In [37]:
weekend_orders

,DayofWeek,OrderID,OtotSubTotal,OrdServiceType,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,...,windspeed,winddir,sealevelpressure,cloudcover,visibility,solarradiation,solarenergy,uvindex,severerisk,conditions
BusDate,,,,,,,,,,,,,,,,,,,,,
1/13/2023,Friday,1,13.43,1,38,31,40,31.9,21.6,27.2,...,14.8,340.6,1017.5,100.0,9.2,13.3,1.2,1,10.0,"Snow, Rain, Overcast"
1/13/2023,Friday,2,28.49,1,38,31,40,31.9,21.6,27.2,...,14.8,340.6,1017.5,100.0,9.2,13.3,1.2,1,10.0,"Snow, Rain, Overcast"
1/13/2023,Friday,3,15.68,1,38,31,40,31.9,21.6,27.2,...,14.8,340.6,1017.5,100.0,9.2,13.3,1.2,1,10.0,"Snow, Rain, Overcast"
1/13/2023,Friday,5,23.92,1,38,31,40,31.9,21.6,27.2,...,14.8,340.6,1017.5,100.0,9.2,13.3,1.2,1,10.0,"Snow, Rain, Overcast"
1/13/2023,Friday,6,22.57,2,38,31,40,31.9,21.6,27.2,...,14.8,340.6,1017.5,100.0,9.2,13.3,1.2,1,10.0,"Snow, Rain, Overcast"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9/5/2021,Sunday,90,24.07,1,80,62,70,81.8,61.8,71.5,...,10.7,284.2,1011.8,73.3,8.0,47.2,4.1,1,NaN,"Rain, Partially cloudy"
9/5/2021,Sunday,93,108.21,1,80,62,70,81.8,61.8,71.5,...,10.7,284.2,1011.8,73.3,8.0,47.2,4.1,1,NaN,"Rain, Partially cloudy"
9/5/2021,Sunday,96,7.99,1,80,62,70,81.8,61.8,71.5,...,10.7,284.2,1011.8,73.3,8.0,47.2,4.1,1,NaN,"Rain, Partially cloudy"


In [38]:
# Checking the correlation of temperature on Weekend days

#fri_orders_per_avgtemp = friday_orders.temp.value_counts()
#fri_orders_per_avgtemp

montues_orders_per_avgtemp = montues_orders.temp.value_counts()
montues_orders_per_avgtemp

temp
70    4956
60    3498
50    3115
80    3036
40    2571
30    1657
20     312
Name: count, dtype: int64

In [40]:
# Correlation matrix for the combined dataframe

print(combined_dataset.corr())
#corr_matrix

#Correlation matrix first try did not work - need to dig into previous class examples and find this to see if I can figure this out. Ending for the day
# Second try got closer - maybe try removing the dayofthe week column and try that.

ValueError: could not convert string to float: 'Saturday'